In [1]:
import sagemaker

In [2]:
session = sagemaker.Session()
role = sagemaker.get_execution_role()

# Push data to S3 bucket

In [5]:
DATA_FILE = './data/iris.csv'
bucket_name = 'rsjainaimlmodels'
prefix = 'iris/data'
s3_output_path = 's3://'+bucket_name

print ('------------>uploading data to s3')
s3_data_path = session.upload_data(path=DATA_FILE, 
                                   bucket=bucket_name, 
                                   key_prefix=prefix)
print ('------------>data uploaded to -', s3_data_path)
print ('------------>model would be uploaded to -', s3_output_path)

------------>uploading data to s3
------------>data uploaded to - s3://rsjainaimlmodels/iris/data/iris.csv
------------>model would be uploaded to - s3://rsjainaimlmodels


# Create Sklearn Executor

In [9]:
from sagemaker.sklearn import SKLearn

executor = SKLearn(entry_point='train.py',
                  train_instance_type='ml.c4.xlarge',
                  sagemaker_session = session,
                  role = role,
                  hyperparameters = {'max_iter':15,
                                    'class_weight':'balanced'},
                  output_path = s3_output_path)

In [10]:
executor.fit(s3_data_path)

2020-05-09 18:33:38 Starting - Starting the training job...
2020-05-09 18:33:40 Starting - Launching requested ML instances......
2020-05-09 18:34:45 Starting - Preparing the instances for training...
2020-05-09 18:35:35 Downloading - Downloading input data...
2020-05-09 18:36:07 Training - Downloading the training image...
2020-05-09 18:36:39 Uploading - Uploading generated training model
2020-05-09 18:36:39 Completed - Training job completed
2020-05-09 18:36:26,963 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-05-09 18:36:26,966 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-05-09 18:36:26,976 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-05-09 18:36:27,258 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-05-09 18:36:27,258 sagemaker-containers INFO     Generating setup.cfg
2020-05-09 18:36:27,258 sagemaker-containers INF

# Deploy Model

In [11]:
predictor = executor.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
print('\nModel Deployed!')

print (predictor.endpoint)

---------------!
Model Deployed!
sagemaker-scikit-learn-2020-05-09-18-33-38-557


# Predict

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv(DATA_FILE,engine='python')
X = data.iloc[:,1:5]
y = data.iloc[:,5]

train_x, test_x, train_y, test_y = train_test_split(X,y)
pred_y = predictor.predict(test_x.values)
print (pred_y)
print (test_y)

[2. 2. 2. 2. 1. 1. 1. 1. 2. 0. 1. 1. 2. 1. 1. 2. 1. 0. 2. 0. 0. 1. 0. 2.
 2. 0. 1. 2. 0. 0. 0. 2. 2. 1. 0. 1. 0. 0.]
127    2.0
149    2.0
139    2.0
146    2.0
61     1.0
65     1.0
90     1.0
74     1.0
101    2.0
15     0.0
68     1.0
79     1.0
110    2.0
95     1.0
75     1.0
125    2.0
62     1.0
5      0.0
144    2.0
20     0.0
34     0.0
51     1.0
39     0.0
104    2.0
111    2.0
9      0.0
89     1.0
140    2.0
18     0.0
21     0.0
48     0.0
128    2.0
84     1.0
92     1.0
30     0.0
96     1.0
4      0.0
7      0.0
Name: class, dtype: float64
